<a href="https://colab.research.google.com/github/Roby290597/nlp_exercise/blob/main/bundestags_reden_analyse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
import os 
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET


In [10]:
print(os.listdir())

['.git', '21032.xml', 'bundestags_reden_analyse.ipynb', 'bundestag_aktuell.txt', 'bundestag_aktuell.xml', 'sentiment_analyse.ipynb']


In [3]:
# import xml.etree.ElementTree as ET
# tree = ET.parse('bundestag_21019.xml')
# root = tree.getroot()

In [11]:
import requests

url = "https://www.bundestag.de/resource/blob/1115000/21032.xml" ### Bundestag Reden XML Datei vom November 2025
response = requests.get(url)

with open("21032.xml", "wb") as f:
    f.write(response.content)


In [12]:
import xml.etree.ElementTree as ET

url = "https://www.bundestag.de/resource/blob/1115000/21032.xml" ### Bundestag Reden XML Datei vom November 2025
response = requests.get(url)

# XML in einen Tree parsen
tree = ET.ElementTree(ET.fromstring(response.content))

# Wurzel-Element abrufen
root = tree.getroot()

print(root.keys())

['vertrieb', 'herstellung', 'sitzung-ort', 'herausgeber', 'issn', 'wahlperiode', 'sitzung-nr', 'sitzung-datum', 'sitzung-start-uhrzeit', 'sitzung-ende-uhrzeit', 'sitzung-naechste-datum', 'start-seitennr']


In [13]:
root.attrib

{'vertrieb': 'Bundesanzeiger Verlag GmbH, Postfach 1 0 05 34, 50445 Köln, Telefon (02 21) 97 66 83 40, Fax (02 21) 97 66 83 44, www.bundesanzeiger-verlag.de',
 'herstellung': 'H. Heenemann GmbH  Co. KG, Buch- und Offsetdruckerei, Bessemerstraße 83–91, 12103 Berlin, www.heenemann-druck.de',
 'sitzung-ort': 'Berlin',
 'herausgeber': 'Deutscher Bundestag',
 'issn': '0722-7980',
 'wahlperiode': '21',
 'sitzung-nr': '32',
 'sitzung-datum': '10.10.2025',
 'sitzung-start-uhrzeit': '09:00',
 'sitzung-ende-uhrzeit': '15:13',
 'sitzung-naechste-datum': '15.10.2025',
 'start-seitennr': '3443'}

In [14]:
root[2][1][1].attrib    

{'anlagen-typ': 'Entschuldigte Abgeordnete'}

In [15]:
# Extrahiere Rednernamen und alle zugehörigen Redetexte aus dem XML, wie sie in <rede> gespeichert sind
def extract_speeches(root):
    speeches = []
    for rede in root.findall(".//rede"):
        # Rednername aus dem ersten <p klasse="redner"> extrahieren
        redner_p = rede.find("./p[@klasse='redner']")
        if redner_p is not None:
            # Versuche Vorname und Nachname aus verschachtelten Tags zu holen
            name_tag = redner_p.find(".//name")
            if name_tag is not None:
                vorname = name_tag.findtext("vorname", default="").strip()
                nachname = name_tag.findtext("nachname", default="").strip()
                redner_name = f"{vorname} {nachname}".strip()
            else:
                # Fallback: Text direkt aus <p> nehmen
                redner_name = redner_p.text.split(":")[0].strip() if redner_p.text else ""
        else:
            redner_name = ""

        # Alle <p> mit Redeinhalt (ohne klasse="redner" und ohne <kommentar>)
        rede_text = []
        for p in rede.findall("./p"):
            if p.get("klasse") != "redner" and p.text and p.text.strip():
                rede_text.append(p.text.strip())
        full_text = "\n".join(rede_text)
        speeches.append({"name": redner_name, "text": full_text})
    return speeches


reden = {}
redner = []
# Beispiel-Ausgabe
for speech in extract_speeches(root):
    #print(f"Redner: {speech['name']}\nRede: {speech['text'][:200]}...\n")
    if speech['name'] not in redner:
        redner.append(speech['name'])
        reden[speech['name']] = speech['text']
    else:
        print("DOPPELT:", speech['name'])
        reden[speech['name']] += "\n Nächste Rede:" + speech['text']

DOPPELT: Mandy Eißing
DOPPELT: Peter Bohnhof
DOPPELT: Mahmut Özdemir
DOPPELT: Klaus Wiener
DOPPELT: Mirze Edis


In [ ]:
reden['Peter Bohnhof']


'Frau Präsidentin! Meine Damen und Herren! Liebe Bürgerinnen und Bürger! Stellen Sie sich vor, Sie schuften jeden Tag, geben alles für Ihren Job, für Ihre Familie, für unser Land, und dann kommt diese Regierung und verspricht Ihnen Schutz, Fairness und mehr Wohlstand. Aber was passiert wirklich? Für diese Regierung gilt nämlich in der ganzen Legislatur der Grundsatz: Erst versprochen, dann gebrochen.\nUnd genau das sehen wir auch bei diesem sogenannten Tariftreuegesetz: ein Gesetz\xa0– ein Wolf im Schafspelz geradezu\xa0–, das allen die Freiheit raubt, ob Arbeitnehmern, Unternehmern oder einfach nur jemandem, der auf faire Chancen hofft. Dieses Thema betrifft uns alle.\nSchauen wir uns deshalb den Gesetzentwurf mal etwas näher an. Die Regierung will bei öffentlichen Aufträgen nur noch Firmen berücksichtigen, die ein sogenanntes Tariftreueversprechen abgeben. Auf den ersten Blick klingt das nach Gerechtigkeit. Die Wahrheit ist eine andere: Es ist ein massiver Eingriff in unsere Wirtscha

In [16]:
# Extrahiere alle Redner aus der <rednerliste> im XML
def extract_all_speakers(root):
    speakers = []
    for rednerliste in root.findall(".//rednerliste"):
        for redner in rednerliste.findall("redner"):
            name_tag = redner.find("name")
            if name_tag is not None:
                vorname = name_tag.findtext("vorname", default="").strip()
                nachname = name_tag.findtext("nachname", default="").strip()
                fraktion = name_tag.findtext("fraktion", default="").strip()
                rolle_lang = name_tag.findtext("rolle/rolle_lang", default="").strip()
                rolle_kurz = name_tag.findtext("rolle/rolle_kurz", default="").strip()
                speakers.append({
                    #"id": redner.get("id"),
                    "vorname": vorname,
                    "nachname": nachname,
                    "fraktion": fraktion,
                    #"rolle_lang": rolle_lang,
                    #"rolle_kurz": rolle_kurz
                })
                
    return speakers

redner_party = []
all_speakers = extract_all_speakers(root)
for speaker in all_speakers:
    print(speaker)
    redner_party.append((speaker['vorname'] + " " + speaker['nachname'], speaker['fraktion']))

{'vorname': 'Bärbel', 'nachname': 'Bas', 'fraktion': ''}
{'vorname': 'Hans-Jürgen', 'nachname': 'Goßner', 'fraktion': 'AfD'}
{'vorname': 'Wilfried', 'nachname': 'Oellers', 'fraktion': 'CDU/CSU'}
{'vorname': 'Ricarda', 'nachname': 'Lang', 'fraktion': 'BÜNDNIS\xa090/DIE GRÜNEN'}
{'vorname': 'Pascal', 'nachname': 'Meiser', 'fraktion': 'Die Linke'}
{'vorname': 'Dagmar', 'nachname': 'Schmidt', 'fraktion': 'SPD'}
{'vorname': 'Peter', 'nachname': 'Bohnhof', 'fraktion': 'AfD'}
{'vorname': 'Sandra', 'nachname': 'Carstensen', 'fraktion': 'CDU/CSU'}
{'vorname': 'Armin', 'nachname': 'Grau', 'fraktion': 'BÜNDNIS\xa090/DIE GRÜNEN'}
{'vorname': 'Nora', 'nachname': 'Seitz', 'fraktion': 'CDU/CSU'}
{'vorname': 'Leif-Erik', 'nachname': 'Holm', 'fraktion': 'AfD'}
{'vorname': 'Peter', 'nachname': 'Aumer', 'fraktion': 'CDU/CSU'}
{'vorname': 'Jan', 'nachname': 'Dieren', 'fraktion': 'SPD'}
{'vorname': 'Elisabeth', 'nachname': 'Winkelmeier-Becker', 'fraktion': 'CDU/CSU'}
{'vorname': 'Tobias Matthias', 'nachnam

In [17]:
from collections import Counter

party_list = [name for name, party in redner_party] 

counter = Counter(party_list)
print(counter)

Counter({'Bärbel Bas': 1, 'Hans-Jürgen Goßner': 1, 'Wilfried Oellers': 1, 'Ricarda Lang': 1, 'Pascal Meiser': 1, 'Dagmar Schmidt': 1, 'Peter Bohnhof': 1, 'Sandra Carstensen': 1, 'Armin Grau': 1, 'Nora Seitz': 1, 'Leif-Erik Holm': 1, 'Peter Aumer': 1, 'Jan Dieren': 1, 'Elisabeth Winkelmeier-Becker': 1, 'Tobias Matthias Peterka': 1, 'Axel Müller': 1, 'Lena Gumnior': 1, 'Mahmut Özdemir': 1, 'Luke Hoß': 1, 'Christian Moser': 1, 'Rainer Galla': 1, 'Maja Wallstein': 1, 'Helge Limburg': 1, 'Johannes Wiegelmann': 1, 'Knuth Meyer-Soltau': 1, 'David Preisendanz': 1, 'Wolfram Weimer': 1, 'Götz Frömming': 1, 'Elisabeth Kaiser': 1, 'Katrin Göring-Eckardt': 1, 'Mandy Eißing': 1, 'Ottilie Klein': 1, 'Sven Wendorf': 1, 'Franziska Kersten': 1, 'Sepp Müller': 1, 'Wolfgang Dahler': 1, 'Birgit Bessin': 1, 'Felix Döring': 1, 'Misbah Khan': 1, 'Konrad Körner': 1, 'Anna Rathert': 1, 'Reem Alabali Radovan': 1, 'Rocco Kever': 1, 'Nicolas Zippelius': 1, 'Claudia Roth': 1, 'Mirze Edis': 1, 'Klaus Wiener': 1, 'Jo

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# TF-IDF-Vektor für die Rede von Bärbel Bas erzeugen

# Rede aus bestehendem Dictionary verwenden
text = reden.get("Bärbel Bas", "")
if not text:
    raise ValueError("Keine Rede für 'Bärbel Bas' im Dictionary 'reden' gefunden.")

# Deutsches Stopword-Set nutzen, TF-IDF berechnen
vectorizer = TfidfVectorizer(stop_words="english")
X = vectorizer.fit_transform([text])  # Ergebnis als sparse matrix
baerbel_vec = X.toarray()[0]          # dichter Vektor

print("Vektor-Länge:", baerbel_vec.shape[0])

# Wichtigste Terme anzeigen (Top 20)
feature_names = vectorizer.get_feature_names_out()
top_n = 20
top_idx = np.argsort(baerbel_vec)[::-1][:top_n]
print("\nTop-Terme (mit TF-IDF-Werten):")
for i in top_idx:
    print(f"{feature_names[i]}: {baerbel_vec[i]:.4f}")


Vektor-Länge: 309

Top-Terme (mit TF-IDF-Werten):
und: 0.3627
auch: 0.3174
wir: 0.2947
die: 0.2720
der: 0.2267
das: 0.2040
ein: 0.1814
für: 0.1587
um: 0.1587
es: 0.1360
hat: 0.1133
sind: 0.1133
ist: 0.1133
dass: 0.0907
wer: 0.0907
den: 0.0907
haben: 0.0907
kollegen: 0.0907
unternehmen: 0.0907
liebe: 0.0907


In [22]:
from transformers import pipeline


from transformers import AutoTokenizer, AutoModelWithLMHead

tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-emotion")

model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-emotion")

def get_emotion(text):
  input_ids = tokenizer.encode(text + '</s>', return_tensors='pt')

  output = model.generate(input_ids=input_ids,
               max_length=2)
  
  dec = [tokenizer.decode(ids) for ids in output]
  label = dec[0]
  return label
  
get_emotion("i feel as if i havent blogged in ages are at least truly blogged i am doing an update cute") # Output: 'joy'
 
get_emotion("i have a feeling i kinda lost my best friend") # Output: 'sadness'
emotion_pipeline = pipeline("text-classification", model="mrm8488/t5-base-finetuned-emotion")

ValueError: Converting from SentencePiece and Tiktoken failed, if a converter for SentencePiece is available, provide a model path with a SentencePiece tokenizer.model file.Currently available slow->fast converters: ['AlbertTokenizer', 'BartTokenizer', 'BarthezTokenizer', 'BertTokenizer', 'BigBirdTokenizer', 'BlenderbotTokenizer', 'CamembertTokenizer', 'CLIPTokenizer', 'CodeGenTokenizer', 'ConvBertTokenizer', 'DebertaTokenizer', 'DebertaV2Tokenizer', 'DistilBertTokenizer', 'DPRReaderTokenizer', 'DPRQuestionEncoderTokenizer', 'DPRContextEncoderTokenizer', 'ElectraTokenizer', 'FNetTokenizer', 'FunnelTokenizer', 'GPT2Tokenizer', 'HerbertTokenizer', 'LayoutLMTokenizer', 'LayoutLMv2Tokenizer', 'LayoutLMv3Tokenizer', 'LayoutXLMTokenizer', 'LongformerTokenizer', 'LEDTokenizer', 'LxmertTokenizer', 'MarkupLMTokenizer', 'MBartTokenizer', 'MBart50Tokenizer', 'MPNetTokenizer', 'MobileBertTokenizer', 'MvpTokenizer', 'NllbTokenizer', 'OpenAIGPTTokenizer', 'PegasusTokenizer', 'Qwen2Tokenizer', 'Realm

In [30]:
# pip install sentence-transformers
from sentence_transformers import SentenceTransformer
import numpy as np

model = SentenceTransformer("all-MiniLM-L6-v2")  # klein & schnell; gute Qualität

def embed(texts):
    # texts: str or list[str]
    return model.encode(texts, convert_to_numpy=True, normalize_embeddings=False)

def cosine_sim(a, b):
    # numerisch stabil: L2-normalisieren dann dot
    a = a / np.linalg.norm(a)
    b = b / np.linalg.norm(b)
    return float(np.dot(a, b))

# Beispiel
t1 = "Das ist ein Beispieltext."
t2 = "Das ist ein Beispieltext."
t2 = "Döner"
t1 = "Pizza"
v1, v2 = embed([t1, t2])
print("Cosine similarity:", cosine_sim(v1, v2))


Cosine similarity: 0.23459944128990173
